# Timer

In [1]:
from time import sleep
from datetime import datetime as dt

start_time = "30.05.2023, 10:20"
start_time = dt.strptime(start_time, "%d.%m.%Y, %H:%M")

while True:
    if start_time < dt.now():
        break
    else:
        sleep(10)

# Parameter optimization

In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

ttype = 'sell'
pattern = ['Pattern', 'Trend', 'Volume24']
indicator_list = pattern
indicator_list_higher = ['Trend', 'MACD']

work_timeframe = '1h'
higher_timeframe = '4h'
opt_limit = 100000
load = False

print(f'Timeframe is {higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = pattern
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'Pattern': {
                            'vol_window': [96], # [12, 24, 36, 48, 60, 72, 84, 96, 108, 120],
                            'vol_q_high': [0.65], # np.arange(0.5, 1, 0.05), 
                            'vol_q_low': [0.15], # np.arange(0.1, 0.5, 0.05),
                            'window_low_bound': [4], 
                            'window_high_bound': [9],
                            'first_candle': [0.5], # np.arange(0.5, 1, 0.05),
                            'second_candle': [0.5], #np.arange(0.5, 1, 0.05)
                            },
                'Trend': {
                            'timeperiod': [30], # np.arange(2, 50, 2),
                            'low_bound': np.arange(0, 0.011, 0.001)
                            }
             }


opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load, 'optimize')

# stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
# if not stat_list:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
# else:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 4h, trade type is sell
Number of combinations is 11


100%|██████████| 11/11 [54:04<00:00, 294.92s/it]


# Check local statistics

In [2]:
# result
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
             pct_price_diff_cols].sort_values('mar_avg')
stat.head(24)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
0,Pattern_Trend_Volume24,96,0.65,0.15,4,9,0.5,0.5,30,0.000,9979,1.058217,-2.461667,580.944117,-24564.971667,1.0918,1.1229,1.1180,1.1096,1.1097,1.1070,1.0889,1.0388,1.0365,1.0322,1.0329,1.0320,1.0449,1.0498,1.0488,1.0494,1.0445,1.0428,1.0372,1.0365,1.0363,1.0296,1.0327,1.0244,-0.23,-0.45,-0.67,-0.89,-1.11,-1.32,-1.53,-1.72,-1.91,-2.10,-2.27,-2.42,-2.58,-2.73,-2.88,-3.05,-3.22,-3.41,-3.61,-3.83,-4.11,-4.26,-4.36,-4.42
1,Pattern_Trend_Volume24,96,0.65,0.15,4,9,0.5,0.5,30,0.001,9921,1.056275,-2.442917,558.304275,-24236.176250,1.0926,1.1366,1.1338,1.1241,1.1241,1.1175,1.0980,1.0475,1.0429,1.0351,1.0279,1.0225,1.0338,1.0399,1.0382,1.0363,1.0328,1.0314,1.0249,1.0246,1.0237,1.0185,1.0247,1.0192,-0.23,-0.45,-0.66,-0.88,-1.10,-1.31,-1.51,-1.71,-1.90,-2.08,-2.25,-2.40,-2.55,-2.70,-2.85,-3.02,-3.20,-3.39,-3.58,-3.80,-4.09,-4.23,-4.34,-4.40
2,Pattern_Trend_Volume24,96,0.65,0.15,4,9,0.5,0.5,30,0.002,9921,1.056275,-2.442917,558.304275,-24236.176250,1.0926,1.1366,1.1338,1.1241,1.1241,1.1175,1.0980,1.0475,1.0429,1.0351,1.0279,1.0225,1.0338,1.0399,1.0382,1.0363,1.0328,1.0314,1.0249,1.0246,1.0237,1.0185,1.0247,1.0192,-0.23,-0.45,-0.66,-0.88,-1.10,-1.31,-1.51,-1.71,-1.90,-2.08,-2.25,-2.40,-2.55,-2.70,-2.85,-3.02,-3.20,-3.39,-3.58,-3.80,-4.09,-4.23,-4.34,-4.40
3,Pattern_Trend_Volume24,96,0.65,0.15,4,9,0.5,0.5,30,0.003,9923,1.056171,-2.441250,557.383179,-24224.523750,1.0926,1.1367,1.1338,1.1240,1.1240,1.1174,1.0980,1.0475,1.0429,1.0351,1.0278,1.0224,1.0337,1.0399,1.0380,1.0361,1.0326,1.0312,1.0247,1.0244,1.0235,1.0184,1.0244,1.0190,-0.23,-0.45,-0.66,-0.88,-1.10,-1.31,-1.51,-1.70,-1.90,-2.08,-2.24,-2.40,-2.55,-2.70,-2.85,-3.02,-3.20,-3.39,-3.58,-3.80,-4.08,-4.23,-4.34,-4.39
4,Pattern_Trend_Volume24,96,0.65,0.15,4,9,0.5,0.5,30,0.004,9923,1.056171,-2.441250,557.383179,-24224.523750,1.0926,1.1367,1.1338,1.1240,1.1240,1.1174,1.0980,1.0475,1.0429,1.0351,1.0278,1.0224,1.0337,1.0399,1.0380,1.0361,1.0326,1.0312,1.0247,1.0244,1.0235,1.0184,1.0244,1.0190,-0.23,-0.45,-0.66,-0.88,-1.10,-1.31,-1.51,-1.70,-1.90,-2.08,-2.24,-2.40,-2.55,-2.70,-2.85,-3.02,-3.20,-3.39,-3.58,-3.80,-4.08,-4.23,-4.34,-4.39
5,Pattern_Trend_Volume24,96,0.65,0.15,4,9,0.5,0.5,30,0.005,9923,1.056171,-2.441250,557.383179,-24224.523750,1.0926,1.1367,1.1338,1.1240,1.1240,1.1174,1.0980,1.0475,1.0429,1.0351,1.0278,1.0224,1.0337,1.0399,1.0380,1.0361,1.0326,1.0312,1.0247,1.0244,1.0235,1.0184,1.0244,1.0190,-0.23,-0.45,-0.66,-0.88,-1.10,-1.31,-1.51,-1.70,-1.90,-2.08,-2.24,-2.40,-2.55,-2.70,-2.85,-3.02,-3.20,-3.39,-3.58,-3.80,-4.08,-4.23,-4.34,-4.39
6,Pattern_Trend_Volume24,96,0.65,0.15,4,9,0.5,0.5,30,0.006,9923,1.056171,-2.441250,557.383179,-24224.523750,1.0926,1.1367,1.1338,1.1240,1.1240,1.1174,1.0980,1.0475,1.0429,1.0351,1.0278,1.0224,1.0337,1.0399,1.0380,1.0361,1.0326,1.0312,1.0247,1.0244,1.0235,1.0184,1.0244,1.0190,-0.23,-0.45,-0.66,-0.88,-1.10,-1.31,-1.51,-1.70,-1.90,-2.08,-2.24,-2.40,-2.55,-2.70,-2.85,-3.02,-3.20,-3.39,-3.58,-3.80,-4.08,-4.23,-4.34,-4.39
7,Pattern_Trend_Volume24,96,0.65,0.15,4,9,0.5,0.5,30,0.007,9924,1.056196,-2.441250,557.687450,-24226.965000,1.0926,1.1367,1.1338,1.124

In [3]:
# vol_q
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg')
stat.head(20)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
254,Pattern_Trend,48,0.80,0.20,1,5,0.5,0.5,24,0,96,0.711417,-11.187917,-27.704000,-1074.040000,0.8253,0.7156,0.7507,0.7291,0.7378,0.7441,0.7626,0.7630,0.7282,0.7121,0.6781,0.6758,0.6990,0.6880,0.6928,0.6706,0.6710,0.6708,0.6733,0.6861,0.6923,0.6985,0.7012,0.7080,-0.19,-0.38,-0.64,-0.99,-1.42,-2.01,-2.84,-3.95,-5.51,-8.36,-10.93,-13.14,-13.39,-13.51,-13.62,-14.92,-16.05,-17.36,-18.65,-19.94,-21.17,-22.35,-23.37,-23.82
308,Pattern_Trend,48,0.95,0.20,1,5,0.5,0.5,24,0,77,0.826483,-11.010000,-13.360783,-847.770000,0.8634,0.7353,0.8682,0.8687,0.9764,0.9599,0.9738,0.9812,0.8947,0.9007,0.8368,0.8079,0.8164,0.7878,0.7910,0.7507,0.7505,0.7448,0.7435,0.7518,0.7562,0.7592,0.7529,0.7638,0.01,0.03,-0.07,-0.27,-0.62,-1.14,-1.97,-3.13,-4.85,-8.15,-11.09,-13.56,-13.62,-13.50,-13.39,-14.75,-15.90,-17.28,-18.64,-20.04,-21.37,-22.72,-23.87,-24.35
290,Pattern_Trend,48,0.90,0.20,1,5,0.5,0.5,24,0,81,0.822413,-10.439167,-14.384587,-845.572500,0.8527,0.7334,0.8594,0.8550,0.9535,0.9404,0.9527,0.9597,0.8778,0.8796,0.8201,0.7950,0.8054,0.7889,0.7943,0.7551,0.7557,0.7556,0.7551,0.7653,0.7694,0.7721,0.7656,0.7761,0.00,0.03,-0.06,-0.24,-0.56,-1.05,-1.83,-2.93,-4.55,-7.68,-10.47,-12.81,-12.88,-12.78,-12.68,-13.99,-15.09,-16.42,-17.71,-19.07,-20.31,-21.60,-22.69,-23.17
236,Pattern_Trend,48,0.75,0.20,1,5,0.5,0.5,24,0,108,0.794242,-10.383333,-22.221900,-1121.400000,0.8340,0.7883,0.8356,0.8244,0.8089,0.8139,0.8356,0.8303,0.8150,0.8056,0.7636,0.7637,0.7906,0.7768,0.7859,0.7614,0.7571,0.7595,0.7651,0.7783,0.7852,0.7904,0.7928,0.7998,-0.16,-0.34,-0.59,-0.89,-1.27,-1.81,-2.58,-3.59,-5.03,-7.63,-9.98,-12.02,-12.29,-12.48,-12.65,-13.88,-14.96,-16.19,-17.40,-18.58,-19.73,-20.89,-21.88,-22.38
272,Pattern_Trend,48,0.85,0.20,1,5,0.5,0.5,24,0,92,0.782821,-9.827500,-19.980483,-904.130000,0.7544,0.6746,0.7819,0.7762,0.8656,0.8651,0.8850,0.8796,0.8250,0.8353,0.7884,0.7682,0.7817,0.7653,0.7694,0.7410,0.7424,0.7425,0.7439,0.7585,0.7613,0.7637,0.7573,0.7614,-0.00,-0.00,-0.09,-0.28,-0.58,-1.06,-1.79,-2.78,-4.28,-7.12,-9.68,-11.86,-12.01,-12.01,-12.02,-13.26,-14.32,-15.56,-16.77,-18.02,-19.13,-20.29,-21.26,-21.69
255,Pattern_Trend,48,0.80,0.25,1,5,0.5,0.5,24,0,117,0.753004,-9.456667,-28.898513,-1106.430000,0.8485,0.7822,0.8030,0.7886,0.7814,0.7851,0.7904,0.7953,0.7664,0.7492,0.7305,0.7212,0.7398,0.7235,0.7297,0.7097,0.7141,0.7156,0.7152,0.7258,0.7302,0.7358,0.7425,0.7484,-0.16,-0.31,-0.52,-0.80,-1.16,-1.67,-2.37,-3.29,-4.61,-6.98,-9.16,-11.03,-11.28,-11.43,-11.57,-12.69,-13.66,-14.77,-15.84,-16.92,-17.88,-18.91,-19.76,-20.19
218,Pattern_Trend,48,0.70,0.20,1,5,0.5,0.5,24,0,125,0.822329,-9.442083,-22.208854,-1180.260417,0.8584,0.8163,0.8509,0.8183,0.8125,0.8230,0.8518,0.8706,0.8399,0.8170,0.7826,0.7859,0.8235,0.8150,0.8310,0.8027,0.7981,0.7900,0.8007,0.8145,0.8231,0.8323,0.8336,0.8442,-0.23,-0.43,-0.66,-0.93,-1.31,-1.81,-2.50,-3.41,-4.69,-6.96,-9.01,-10.81,-11.08,-11.28,-11.49,-12.59,-13.56,-14.68,-15.76,-16.88,-17.90,-18.76,-19.68,-20.20
200,Pattern_Trend,48,0.65,0.20,1,5,0.5,0.5,24,0,146,0.855958,-9.045417,-21.030083,-1320.630833,0.9061,0.8198,0.8472,0.8216,0.8388,0.8525,0

In [4]:
# candles
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_rank')
stat.head(20)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
11,Pattern_Trend,48,0.8,0.2,1,5,0.55,0.55,24,0,85,0.770771,-13.243750,-19.484479,-1125.718750,0.8585,0.7758,0.8170,0.7713,0.7767,0.7738,0.8269,0.8324,0.7919,0.7818,0.7366,0.7280,0.7579,0.7386,0.7407,0.7224,0.7305,0.7285,0.7385,0.7597,0.7696,0.7776,0.7782,0.7856,-0.28,-0.57,-0.93,-1.38,-1.92,-2.64,-3.62,-4.93,-6.75,-10.01,-12.95,-15.47,-15.76,-15.91,-16.02,-17.52,-18.84,-20.34,-21.86,-23.41,-24.90,-26.28,-27.51,-28.05
1,Pattern_Trend,48,0.8,0.2,1,5,0.50,0.55,24,0,91,0.717312,-12.159583,-25.724563,-1106.522083,0.8194,0.7139,0.7493,0.7269,0.7333,0.7322,0.7726,0.7866,0.7434,0.7161,0.6788,0.6741,0.6988,0.6871,0.6907,0.6767,0.6838,0.6778,0.6821,0.7008,0.7093,0.7169,0.7194,0.7255,-0.24,-0.51,-0.83,-1.24,-1.73,-2.39,-3.30,-4.50,-6.18,-9.20,-11.92,-14.25,-14.51,-14.63,-14.73,-16.11,-17.32,-18.70,-20.08,-21.51,-22.85,-24.13,-25.24,-25.73
10,Pattern_Trend,48,0.8,0.2,1,5,0.55,0.50,24,0,92,0.747308,-11.902500,-23.247633,-1095.030000,0.8495,0.7874,0.8181,0.7708,0.7774,0.7649,0.7915,0.7834,0.7552,0.7573,0.7193,0.7147,0.7420,0.7253,0.7288,0.7025,0.7029,0.7013,0.7035,0.7171,0.7233,0.7295,0.7310,0.7387,-0.21,-0.44,-0.73,-1.11,-1.58,-2.21,-3.09,-4.26,-5.91,-8.91,-11.61,-13.93,-14.20,-14.34,-14.47,-15.84,-17.05,-18.42,-19.79,-21.17,-22.50,-23.74,-24.84,-25.31
12,Pattern_Trend,48,0.8,0.2,1,5,0.55,0.60,24,0,83,0.895254,-13.108333,-8.693904,-1087.991667,0.9147,0.8204,0.9030,0.8365,0.9433,0.9171,0.9686,0.9604,0.8899,0.9287,0.8860,0.8833,0.9135,0.8866,0.8879,0.8553,0.8634,0.8588,0.8672,0.8847,0.8973,0.9054,0.9024,0.9117,-0.23,-0.48,-0.80,-1.21,-1.74,-2.45,-3.41,-4.70,-6.51,-9.81,-12.78,-15.32,-15.60,-15.73,-15.85,-17.36,-18.68,-20.18,-21.72,-23.28,-24.81,-26.27,-27.54,-28.14
0,Pattern_Trend,48,0.8,0.2,1,5,0.50,0.50,24,0,96,0.711417,-11.187917,-27.704000,-1074.040000,0.8253,0.7156,0.7507,0.7291,0.7378,0.7441,0.7626,0.7630,0.7282,0.7121,0.6781,0.6758,0.6990,0.6880,0.6928,0.6706,0.6710,0.6708,0.6733,0.6861,0.6923,0.6985,0.7012,0.7080,-0.19,-0.38,-0.64,-0.99,-1.42,-2.01,-2.84,-3.95,-5.51,-8.36,-10.93,-13.14,-13.39,-13.51,-13.62,-14.92,-16.05,-17.36,-18.65,-19.94,-21.17,-22.35,-23.37,-23.82
2,Pattern_Trend,48,0.8,0.2,1,5,0.50,0.60,24,0,89,0.850300,-11.996250,-13.323300,-1067.666250,0.8807,0.7631,0.8402,0.7901,0.8970,0.8734,0.9126,0.9205,0.8479,0.8765,0.8425,0.8417,0.8665,0.8477,0.8507,0.8233,0.8310,0.8207,0.8218,0.8374,0.8479,0.8556,0.8550,0.8634,-0.19,-0.40,-0.66,-1.02,-1.51,-2.15,-3.04,-4.22,-5.88,-8.95,-11.70,-14.05,-14.30,-14.41,-14.52,-15.92,-17.12,-18.52,-19.92,-21.37,-22.76,-24.12,-25.30,-25.88
22,Pattern_Trend,48,0.8,0.2,1,5,0.60,0.60,24,0,77,0.821637,-13.321667,-13.733913,-1025.768333,0.8696,0.8242,0.8612,0.7948,0.8979,0.8625,0.9002,0.8851,0.8216,0.8440,0.7996,0.7889,0.8144,0.7879,0.7954,0.7686,0.7796,0.7761,0.7856,0.8053,0.8191,0.8212,0.8083,0.8082,-0.21,-0.46,-0.75,-1.15,-1.68,-2.38,-3.34,-4.66,-6.55,-10.03,-13.13,-15.77,-15.97,-16.03,-16.09,-17.67,-19.01,-20.60,-22.19,-23.83,-25.17,-26.66,-27.93,-28.46
21,Pattern_Trend,48,0.8,0.2,1,5,0.60,0.55,24,0,84,0.782058,-12.191250,-18.307100,-1024.065000,0.7925,0.7870,0.8043,0.7629,0.8306,0.8189,

In [3]:
# window
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg')
stat.head(20)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
4,Pattern_Trend,72,0.8,0.2,1,5,0.55,0.6,24,0,84,1.070537,-14.191250,5.925150,-1192.065000,1.1035,1.0384,1.1065,1.0375,1.1275,1.0910,1.1023,1.1194,1.1031,1.1150,1.1031,1.0918,1.1018,1.0724,1.0640,1.0440,1.0315,0.9964,0.9969,1.0154,1.0349,1.0447,1.0703,1.0815,-0.35,-0.72,-1.17,-1.70,-2.32,-3.12,-4.16,-5.55,-7.50,-10.84,-13.87,-16.50,-16.85,-17.09,-17.30,-18.87,-20.18,-21.65,-23.24,-24.77,-26.40,-27.69,-29.04,-29.71
3,Pattern_Trend,48,0.8,0.2,1,5,0.55,0.6,24,0,83,0.895254,-13.108333,-8.693904,-1087.991667,0.9147,0.8204,0.9030,0.8365,0.9433,0.9171,0.9686,0.9604,0.8899,0.9287,0.8860,0.8833,0.9135,0.8866,0.8879,0.8553,0.8634,0.8588,0.8672,0.8847,0.8973,0.9054,0.9024,0.9117,-0.23,-0.48,-0.80,-1.21,-1.74,-2.45,-3.41,-4.70,-6.51,-9.81,-12.78,-15.32,-15.60,-15.73,-15.85,-17.36,-18.68,-20.18,-21.72,-23.28,-24.81,-26.27,-27.54,-28.14
5,Pattern_Trend,96,0.8,0.2,1,5,0.55,0.6,24,0,86,0.978942,-13.093333,-1.811017,-1126.026667,0.8950,0.8648,0.9372,0.8917,1.0069,1.0108,1.0111,1.0403,1.0047,1.0288,1.0166,1.0197,1.0307,1.0032,0.9949,0.9721,0.9715,0.9383,0.9366,0.9498,0.9670,0.9795,1.0037,1.0197,-0.26,-0.56,-0.92,-1.39,-1.96,-2.68,-3.66,-4.95,-6.80,-9.98,-12.86,-15.38,-15.74,-15.91,-16.08,-17.52,-18.72,-20.07,-21.52,-22.98,-24.45,-25.61,-26.83,-27.41
0,Pattern_Trend,6,0.8,0.2,1,5,0.55,0.6,24,0,90,0.824008,-4.294167,-15.839250,-386.475000,0.9088,0.8111,0.7254,0.7148,0.7432,0.7329,0.7708,0.8175,0.7756,0.7899,0.7675,0.7702,0.8345,0.8290,0.8679,0.8448,0.8513,0.8535,0.8649,0.8851,0.8895,0.9058,0.9006,0.9216,-0.30,-0.55,-0.81,-1.09,-1.38,-1.70,-2.02,-2.37,-2.66,-3.02,-3.42,-3.89,-4.38,-4.78,-5.22,-5.66,-6.12,-6.57,-7.07,-7.34,-7.79,-7.85,-8.24,-8.83
2,Pattern_Trend,24,0.8,0.2,1,5,0.55,0.6,24,0,75,0.760421,-1.850417,-17.968438,-138.781250,0.9577,0.9485,0.9338,0.8580,0.8896,0.8387,0.8159,0.8490,0.8038,0.7673,0.7097,0.6949,0.7096,0.6887,0.6899,0.6833,0.6919,0.6725,0.6671,0.6698,0.6786,0.6803,0.6718,0.6797,-0.02,-0.08,-0.18,-0.31,-0.42,-0.57,-0.73,-0.93,-1.08,-1.28,-1.53,-1.81,-2.07,-2.30,-2.49,-2.71,-2.92,-3.08,-3.26,-2.99,-3.12,-3.36,-3.43,-3.74
1,Pattern_Trend,12,0.8,0.2,1,5,0.55,0.6,24,0,84,0.800133,-1.234167,-16.788800,-103.670000,1.1565,1.0509,1.0004,0.8903,0.8198,0.7819,0.7957,0.8236,0.7744,0.7491,0.7048,0.7037,0.7391,0.7304,0.7500,0.7325,0.7328,0.7292,0.7311,0.7476,0.7586,0.7641,0.7665,0.7702,-0.18,-0.26,-0.36,-0.48,-0.58,-0.69,-0.81,-0.87,-0.92,-0.98,-1.06,-1.15,-1.27,-1.37,-1.51,-1.67,-1.87,-2.04,-2.24,-1.66,-1.73,-1.90,-1.88,-2.14


In [2]:
# LR timeperiod
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg')
stat.head(20)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
18,Pattern_Trend,48,0.8,0.2,1,5,0.55,0.6,38,0,59,1.169083,-16.797500,9.975917,-991.052500,0.9994,0.8377,1.0009,0.9983,1.1429,1.1181,1.2343,1.2061,1.1530,1.1932,1.2112,1.1968,1.2347,1.2146,1.2138,1.1796,1.2044,1.1973,1.2082,1.2240,1.2390,1.2599,1.2803,1.3103,-0.26,-0.49,-0.84,-1.30,-1.97,-2.82,-4.04,-5.72,-8.18,-12.65,-16.69,-20.11,-20.34,-20.32,-20.27,-22.20,-23.80,-25.72,-27.75,-29.70,-31.93,-33.87,-35.71,-36.46
16,Pattern_Trend,48,0.8,0.2,1,5,0.55,0.6,34,0,65,1.104463,-16.521667,6.790063,-1073.908333,0.8832,0.8220,1.0085,0.9975,1.1360,1.0989,1.2060,1.1792,1.0767,1.1166,1.0853,1.0875,1.1287,1.1192,1.1233,1.0980,1.1232,1.1158,1.1308,1.1458,1.1674,1.1976,1.2164,1.2435,-0.31,-0.63,-1.04,-1.55,-2.26,-3.13,-4.35,-5.97,-8.28,-12.43,-16.17,-19.35,-19.66,-19.80,-19.91,-21.80,-23.42,-25.30,-27.30,-29.17,-31.28,-33.09,-34.79,-35.53
17,Pattern_Trend,48,0.8,0.2,1,5,0.55,0.6,36,0,61,1.154517,-16.365417,9.425517,-998.290417,0.9960,0.8382,1.0209,1.0193,1.1605,1.1345,1.2487,1.2194,1.1084,1.1496,1.1676,1.1562,1.1933,1.1762,1.1759,1.1472,1.1771,1.1745,1.1852,1.2017,1.2234,1.2555,1.2751,1.3040,-0.24,-0.47,-0.81,-1.26,-1.90,-2.74,-3.93,-5.56,-7.94,-12.27,-16.18,-19.49,-19.72,-19.72,-19.71,-21.59,-23.18,-25.08,-27.08,-29.01,-31.19,-33.10,-34.92,-35.68
15,Pattern_Trend,48,0.8,0.2,1,5,0.55,0.6,32,0,66,1.065029,-16.058333,4.291925,-1059.850000,0.8594,0.8060,0.9874,0.9786,1.1234,1.0885,1.1930,1.1592,1.0558,1.0875,1.0325,1.0368,1.0768,1.0701,1.0744,1.0522,1.0770,1.0709,1.0855,1.0999,1.1179,1.1465,1.1326,1.1488,-0.29,-0.58,-0.98,-1.47,-2.15,-3.00,-4.18,-5.76,-8.01,-12.08,-15.74,-18.85,-19.15,-19.26,-19.36,-21.19,-22.78,-24.62,-26.56,-28.39,-30.42,-32.19,-33.84,-34.55
14,Pattern_Trend,48,0.8,0.2,1,5,0.55,0.6,30,0,68,1.001425,-15.434583,0.096900,-1049.551667,0.8893,0.8306,0.9970,0.9475,1.0833,1.0505,1.1499,1.1190,1.0166,1.0465,0.9937,0.9882,1.0209,0.9838,0.9885,0.9633,0.9721,0.9686,0.9754,0.9885,1.0042,1.0189,1.0110,1.0269,-0.29,-0.59,-0.98,-1.45,-2.11,-2.94,-4.08,-5.62,-7.79,-11.72,-15.25,-18.25,-18.50,-18.58,-18.64,-20.39,-21.89,-23.63,-25.46,-27.20,-29.08,-30.76,-32.29,-32.94
23,Pattern_Trend,48,0.8,0.2,1,5,0.55,0.6,48,0,59,0.932704,-15.143750,-3.970454,-893.481250,1.0264,0.7896,0.8448,0.8566,0.9374,0.8919,0.9706,0.9370,0.8927,0.8970,0.9121,0.9131,0.9488,0.9391,0.9443,0.9228,0.9346,0.9319,0.9482,0.9604,0.9696,0.9795,0.9987,1.0378,-0.21,-0.38,-0.65,-1.05,-1.62,-2.34,-3.45,-4.99,-7.28,-11.51,-15.30,-18.49,-18.57,-18.42,-18.25,-20.03,-21.49,-23.24,-25.03,-26.83,-28.79,-30.56,-32.18,-32.79
22,Pattern_Trend,48,0.8,0.2,1,5,0.55,0.6,46,0,60,0.927167,-15.009167,-4.370000,-900.550000,0.9995,0.7749,0.8498,0.8461,0.9240,0.8808,0.9618,0.9302,0.8882,0.8990,0.9138,0.9130,0.9456,0.9357,0.9329,0.9122,0.9263,0.9270,0.9432,0.9554,0.9664,0.9831,1.0022,1.0409,-0.20,-0.37,-0.66,-1.05,-1.60,-2.32,-3.41,-4.93,-7.20,-11.38,-15.14,-18.30,-18.39,-18.26,-18.10,-19.85,-21.28,-23.01,-24.81,-26.60,-28.55,-30.32,-31.94,-32.55
13,Pattern_Trend,48,0.8,0.2,1,5,0.55,0.6,28,0,74,0.990025,-14.672917,-0.738150,-1085.795833,0.9175,0.8512,1.0009,0.9411,1.0541,1.0134,1.1091,1.0903,0.

# Research parameter influence

In [8]:
param = 'Pattern_second_candle'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Pattern_second_candle,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,0.5,0.854897,-0.304701,-168.318122,-286.847375,68130
1,0.6,0.850164,-0.288188,-150.008786,-258.243681,60370
2,0.7,0.849519,-0.281743,-133.779457,-224.315035,52007
3,0.8,0.839842,-0.299785,-109.302578,-205.237062,41617
4,0.9,0.871600,-0.322160,-72.804460,-170.303528,30776


# Plot result in file

In [11]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = param, 'e_ratio_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig(f"{'_'.join(pattern)}'_'{ttype}_{param}", dpi=fig.dpi)

# Save new config data to config file

In [10]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['Pattern', 'LinearReg']
work_timeframe = '1h'
higher_timeframe = '4h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'Pattern': {
                            'vol_window': [48],
                            'vol_q_high': [0.8],
                            'vol_q_low': [0.2],
                            'window_low_bound': [1], 
                            'window_high_bound': [6],
                            'first_candle': [0.55],
                            'second_candle': [0.6],
                            },
                'Trend': {
                            'timeperiod': [36],
                            'low_bound': [0]
                            }
             }
 
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)